In [1]:
import numpy as np
import time
import matplotlib.pyplot as plt
import os
import json
import gym
import warnings

import sys
import time
from IPython.display import display, clear_output
import torch

from cryoenv.mqtt import SoftActorCritic, ReturnTracker, ReplayBuffer, check, subscribe, publish, connect_mqtt
from cryoenv.envs import CryoEnvSigWrapper

from mqtt_protocol import *
from on_message_notebook import receive_as_control

In [ ]:
np.random.seed(rseed)

In [ ]:
warnings.simplefilter('ignore')

In [ ]:
client_id = 'control-secondary'

In [ ]:
env = gym.make('cryoenv:cryoenv-sig-v0',
                   omega=omega,
                   sample_pars=False,
                   pars={'store_raw': False,
                         'max_buffer_len': buffer_size,
                         'tpa_queue': tpa_queue,
                         'pileup_prob': pileup_prob,
                         'xi': np.array([xi]),
                         'tau': np.array([tau]),},
               render_mode='human',
                   )

In [ ]:
state, info = env.reset()
action = env.action_space.sample()

In [ ]:
buffer = ReplayBuffer(buffer_size=buffer_size, input_shape=(env.observation_space.shape[0],), 
                      n_actions=env.action_space.shape[0], memmap_loc=path_buffer)

In [ ]:
agent = SoftActorCritic.load(env, path_models)

In [ ]:
buffer.erase()

In [ ]:
userdata = {'agent': agent,
            'env': env,
            'state': state,
            'action': action,
            'buffer': buffer,
            'learning_starts': learning_starts, 
            'path_models': path_models,
            'greedy': False,
            'channel': channel,
            'omega': omega,
            'set_pars_msg': set_pars_msg,
            'subscribe_acknowledge_msg': subscribe_acknowledge_msg,
            'trigger_msg': trigger_msg,
            'acknowledge_msg': acknowledge_msg,
            'adc_range': adc_range,
            'dac_range': dac_range,
            'Ib_range': Ib_range,
           }

client = connect_mqtt(broker, port, client_id, username, password, userdata = userdata)

In [ ]:
subscribe(client, subscribe_acknowledge_msg['topic'])
subscribe(client, trigger_msg['topic'])
subscribe(client, acknowledge_msg['topic'])

In [ ]:
client.on_message = receive_as_control

In [ ]:
channel_info = {"SubscribeToChannel": channel}
result = client.publish(subscribe_channel_msg['topic'], json.dumps(channel_info))
check(result)

In [ ]:
# userdata['greedy'] = True

In [ ]:
client.loop_forever()